In [8]:
import pygame, sys, random
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import math
import matplotlib.pyplot as plt
import pylab
import datetime

num_episodes = 3601

obstacleRadius = 10
agentRadius = 10

# get size of state and action from environment

boundaryPos = [80, 80]
boundaryLength = [200,200]
initPosAgent = [boundaryPos[0] + boundaryLength[0] / 2 + 100, boundaryPos[1] + boundaryLength[1] / 2]
initPosGoal =  [boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]

moveObstacles = False
action_size = 8
obsNumber = 4
state_size = 2
obsAngleUnit = 1
# state_size = obsNumber + 1            

In [9]:
# A2C(Advantage Actor-Critic) agent
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.load_model = False
        
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.99
        self.actor_lr = 0.002
        self.critic_lr = 0.005

        # create model for policy network
        self.actor = self.build_actor()
        self.critic = self.build_critic()
        
        if self.load_model:
            self.actor.load_weights("./Practice004_DataSave/Actor_PEARL.h5")
            self.critic.load_weights("./Practice004_DataSave/Critic_PEARL.h5")
#         print self.actor.get_weights()[0]
    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(1, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        actor.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_normal'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy', optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(1, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        critic.add(Dense(self.value_size, activation='linear', kernel_initializer='glorot_normal'))
        critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
#         print state
        policy = self.actor.predict(state, batch_size=1).flatten()
#         print policy
#         print self.action_size
#         print self.actor.get_weights()[0]
#         policy = np.reshape(policy, [1, self.action_size])
        return np.random.choice(self.action_size, 1, p=policy)[0]

    # update policy network every episode
    def train_model(self, state, action, reward, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages = np.zeros((1, self.action_size))

        value = self.critic.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]

        if done:
            advantages[0][action] = reward - value
            target[0][0] = reward
        else:
            advantages[0][action] = reward + self.discount_factor * (next_value) - value
            target[0][0] = reward + self.discount_factor * next_value

        self.actor.fit(state, advantages, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)

In [10]:
def ckWall(xPos, yPos):
    flagWall = 0
    if(xPos < boundaryPos[0]):
        xPos = boundaryPos[0]
        flagWall = -1
    elif(xPos > boundaryPos[0] + boundaryLength[0]):
        xPos = boundaryPos[0] + boundaryLength[0]
        flagWall = -1
    if(yPos < boundaryPos[1]):
        yPos = boundaryPos[1]
        flagWall = -1
    elif(yPos > boundaryPos[1] + boundaryLength[1]):
        yPos = boundaryPos[1] + boundaryLength[1]
        flagWall = -1
#     if math.sqrt((xPos - initPosAgent[0]) ** 2 + (yPos - initPosAgent[1]) ** 2) > boundaryRadius - obstacleRadius:
#         if xPos - initPosAgent[0] != 0:
#             xPos = xPos - (xPos - initPosAgent[0])/abs(xPos - initPosAgent[0])
#         if yPos - initPosAgent[1] != 0:
#             yPos = yPos - (yPos - initPosAgent[1])/abs(yPos - initPosAgent[1])
    return [int(round(xPos)), int(round(yPos)), flagWall]

In [11]:
def stateGenerator(intenPref, distPref):
    returnSum = [intenPref, distPref]
    returnSum = np.reshape(returnSum, [1, state_size])
#     print returnSum
    return returnSum

In [12]:
def rangeFinder(allObsPos, rangeCenter):
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
    index = np.argmin(allObsAgtDistance)
    return index

In [13]:
def takeAction(action):
    xAction = 0
    yAction = 0
    if action == 0:
        xAction = 1
    elif action == 1:
        xAction = 1
        yAction = 1
    elif action == 2:
        xAction = 1
        yAction = -1            
    elif action == 3:
        xAction = -1
        yAction = 1
    elif action == 4:
        xAction = -1
    elif action == 5:
        xAction = -1
        yAction = -1
    elif action == 6:
        yAction = -1
    elif action == 7:
        yAction = 1
#     elif action == 8:
#         xAction = 0
#         yAction = 0
        
    return [xAction, yAction]

In [14]:
pygame.init()
screen = pygame.display.set_mode([300,300])
screen.fill([200, 200, 200])

# make A2C agent
agent = A2CAgent(state_size, action_size)

rList, episodes = [], []
obsAngleIdx = 0
# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
posOffset = 60
obstaclePos[0][0] = int(initPosGoal[0] + posOffset) 
obstaclePos[0][1] = int(initPosGoal[1]) 
obstaclePos[1][0] = int(initPosGoal[0]) 
obstaclePos[1][1] = int(initPosGoal[1] + posOffset) 
obstaclePos[2][0] = int(initPosGoal[0] - posOffset) 
obstaclePos[2][1] = int(initPosGoal[1]) 
obstaclePos[3][0] = int(initPosGoal[0]) 
obstaclePos[3][1] = int(initPosGoal[1] - posOffset) 

for e in range(num_episodes):
    # Initialize
    done = False
    score = 0
#    a = False
#    while not a:
#        initPosAgent[0] = random.randrange(boundaryPos[0] + 1, boundaryPos[0] + boundaryLength[0] - 1)
#        initPosAgent[1] = random.randrange(boundaryPos[1] + 1, boundaryPos[1] + boundaryLength[1] - 1)
#        if(math.sqrt((initPosAgent[0] - initPosGoal[0])**2 + (initPosAgent[1] - initPosGoal[1])**2) > obstacleRadius + agentRadius):
#            a = True
#            for i in range(0,obsNumber):
#                if math.sqrt((initPosAgent[0] - obstaclePos[i][0])**2 + (initPosAgent[1] - obstaclePos[i][1])**2) <= obstacleRadius + agentRadius:
#                    a = False
    pygame.draw.circle(screen, [100, 255, 100], initPosGoal, agentRadius, 0)
    x = initPosAgent[0]
    y = initPosAgent[1]
    print("Episode ", e, "Starts!")
    
    idx = rangeFinder(obstaclePos, [x, y])
    FeatureVec1 = (x - initPosGoal[0])**2 + (y - initPosGoal[1])**2
    FeatureVec2 = 1.0 / (0 + (x - obstaclePos[idx][0])**2 + (y - obstaclePos[idx][1])**2)
    state = stateGenerator(FeatureVec1, FeatureVec2)
    while not done:

        action = agent.get_action(state)

        xMove = 0
        yMove = 0
        
        [xMove, yMove] = takeAction(action)
        wallFlag = 0

        x = x + xMove
        y = y + yMove
        [x,y, wallFlag] = ckWall(x,y)

        collisionFlag = 0
        pygame.draw.circle(screen, [100, 100, 255], [x,y], agentRadius, 0)
        idx = rangeFinder(obstaclePos, [x, y])
        FeatureVec1 = (x - initPosGoal[0])**2 + (y - initPosGoal[1])**2
        FeatureVec2 = 1.0 / (0 + (x - obstaclePos[idx][0])**2 + (y - obstaclePos[idx][1])**2)
        next_state = stateGenerator(FeatureVec1, FeatureVec2)

        if(math.sqrt((x - initPosGoal[0])**2 + (y - initPosGoal[1])**2) <= obstacleRadius + agentRadius):
            print("Goal Reached!")
            collisionFlag = 1
            done = True
        for i in range(0,obsNumber):
            pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
            if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) <= obstacleRadius + agentRadius:
                print("Collision!", idx)
                collisionFlag = -1
                done = True
#         if wallFlag == -1:
#             done = True
        if not done:
#             reward = 0
            reward = -1.0 * ((x - initPosGoal[0])**2 + (y - initPosGoal[1])**2) / 100000.0
#             print reward
        else:
            if collisionFlag == 1:
                reward = 10000
                rList.append(1)
            elif collisionFlag == -1:
                reward = -30
                rList.append(0)
#             if wallFlag == -1:
#                 reward = -30
        
        agent.train_model(state, action, reward, next_state, done)
        
        score += reward
        state = next_state
        if done:
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
            for i in range(0,obsNumber):
#                 obsRadius = random.randrange(agentRadius + obstacleRadius + 1, boundaryRadius)
                obsRadius = posOffset
                obsAngleIdx += obsAngleUnit
                
                obsAngle = (90 * i) * math.pi / 180
#                 obsAngle = 0
                obstaclePos[i][0] = int(initPosGoal[0] + obsRadius * math.cos(obsAngle))
                obstaclePos[i][1] = int(initPosGoal[1] + obsRadius * math.sin(obsAngle))

            episodes.append(e)
        pygame.display.flip()
        screen.fill([200,200,200])
    print(datetime.datetime.now().strftime('%H:%M:%S'), score)
    print "===================================================================================================="
    # save the model
    if e % 50 == 0:
        agent.actor.save_weights("./Practice004_DataSave/Actor_PEARL.h5")
        agent.critic.save_weights("./Practice004_DataSave/Critic_PEARL.h5")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1)                 3         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 16        
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1)                 3         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 2         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
('Episode ', 0, 'Starts!')
('Collision!', 0)
('22:17:24', -69.93207999999996)
('Episod

('Collision!', 0)
('22:21:48', -46.49082000000001)
('Episode ', 41, 'Starts!')
('Collision!', 0)
('22:21:49', -42.622009999999996)
('Episode ', 42, 'Starts!')
('Collision!', 0)
('22:21:58', -170.8617300000001)
('Episode ', 43, 'Starts!')
('Collision!', 0)
('22:21:59', -46.85903)
('Episode ', 44, 'Starts!')
('Collision!', 0)
('22:22:01', -59.319390000000055)
('Episode ', 45, 'Starts!')
('Collision!', 0)
('22:22:02', -44.102680000000014)
('Episode ', 46, 'Starts!')
('Collision!', 3)
('22:22:14', -205.0208800000006)
('Episode ', 47, 'Starts!')
('Collision!', 0)
('22:22:15', -42.53126)
('Episode ', 48, 'Starts!')
('Collision!', 0)
('22:22:17', -60.48588000000002)
('Episode ', 49, 'Starts!')
('Collision!', 0)
('22:22:18', -45.915690000000005)
('Episode ', 50, 'Starts!')
('Collision!', 0)
('22:22:19', -50.56693000000001)
('Episode ', 51, 'Starts!')
('Collision!', 3)
('22:22:50', -821.279619999998)
('Episode ', 52, 'Starts!')
('Collision!', 0)
('22:22:51', -48.71171)
('Episode ', 53, 'Starts!

('Collision!', 1)
('22:33:41', -129.20772000000022)
('Episode ', 87, 'Starts!')
('Collision!', 0)
('22:33:44', -97.62390999999974)
('Episode ', 88, 'Starts!')
('Collision!', 0)
('22:33:45', -49.16532)
('Episode ', 89, 'Starts!')
('Collision!', 0)
('22:33:52', -154.6680599999998)
('Episode ', 90, 'Starts!')
('Collision!', 0)
('22:33:57', -116.41555000000015)
('Episode ', 91, 'Starts!')
('Collision!', 0)
('22:34:05', -180.77667000000034)
('Episode ', 92, 'Starts!')
('Collision!', 0)
('22:34:31', -484.30166999999994)
('Episode ', 93, 'Starts!')
('Collision!', 0)
('22:34:54', -497.6922199999992)
('Episode ', 94, 'Starts!')
('Collision!', 1)
('22:35:20', -532.0468900000028)
('Episode ', 95, 'Starts!')
('Collision!', 0)
('22:35:25', -114.18796000000003)
('Episode ', 96, 'Starts!')
Goal Reached!
('22:35:46', 9698.272909999998)
('Episode ', 97, 'Starts!')
('Collision!', 0)
('22:35:51', -119.72675)
('Episode ', 98, 'Starts!')
('Collision!', 0)
('22:35:54', -68.35848000000001)
('Episode ', 99, '

('Collision!', 3)
('22:42:49', -335.5580100000001)
('Episode ', 133, 'Starts!')
('Collision!', 2)
('22:44:17', -2630.7094000001443)
('Episode ', 134, 'Starts!')
('Collision!', 0)
('22:44:37', -459.7811799999984)
('Episode ', 135, 'Starts!')
('Collision!', 1)
('22:44:58', -517.6916099999999)
('Episode ', 136, 'Starts!')
('Collision!', 0)
('22:45:02', -97.51512000000015)
('Episode ', 137, 'Starts!')
('Collision!', 0)
('22:45:04', -65.22545000000001)
('Episode ', 138, 'Starts!')
('Collision!', 0)
('22:45:16', -255.46427999999995)
('Episode ', 139, 'Starts!')
('Collision!', 0)
('22:45:17', -50.98673000000001)
('Episode ', 140, 'Starts!')
('Collision!', 0)
('22:45:26', -178.00493999999978)
('Episode ', 141, 'Starts!')
('Collision!', 0)
('22:45:57', -738.0783600000018)
('Episode ', 142, 'Starts!')
('Collision!', 3)
('22:46:04', -144.53393999999992)
('Episode ', 143, 'Starts!')
('Collision!', 0)
('22:46:05', -42.33776)
('Episode ', 144, 'Starts!')
('Collision!', 3)
('22:46:37', -802.402029999

('Collision!', 0)
('22:50:01', -50.61136000000002)
('Episode ', 179, 'Starts!')
('Collision!', 0)
('22:50:26', -716.7441800000008)
('Episode ', 180, 'Starts!')
('Collision!', 0)
('22:50:28', -63.96425000000001)
('Episode ', 181, 'Starts!')
('Collision!', 1)
('22:50:38', -198.05145000000036)
('Episode ', 182, 'Starts!')
('Collision!', 1)
('22:50:53', -335.4872299999998)
('Episode ', 183, 'Starts!')
('Collision!', 1)
('22:51:22', -591.7885800000037)
('Episode ', 184, 'Starts!')
Goal Reached!
('22:51:42', 9659.03287)
('Episode ', 185, 'Starts!')
('Collision!', 0)
('22:51:44', -72.08331000000007)
('Episode ', 186, 'Starts!')
('Collision!', 3)
('22:53:28', -3057.321040000092)
('Episode ', 187, 'Starts!')
('Collision!', 0)
('22:53:54', -523.263089999997)
('Episode ', 188, 'Starts!')
('Collision!', 0)
('22:54:02', -186.92413000000022)
('Episode ', 189, 'Starts!')
('Collision!', 1)
('22:54:56', -1265.945409999988)
('Episode ', 190, 'Starts!')
('Collision!', 0)
('22:55:22', -652.8008800000003)


...

('Collision!', 0)
('10:30:58', -3682.120869999991)
('Episode ', 3397, 'Starts!')
('Collision!', 1)
('10:31:03', -95.76964)
('Episode ', 3398, 'Starts!')
('Collision!', 0)
('10:31:05', -50.59091999999998)
('Episode ', 3399, 'Starts!')
('Collision!', 0)
('10:31:06', -61.768860000000004)
('Episode ', 3400, 'Starts!')
('Collision!', 0)
('10:31:09', -69.59713999999997)
('Episode ', 3401, 'Starts!')
('Collision!', 0)
('10:31:09', -43.61428)
('Episode ', 3402, 'Starts!')
('Collision!', 0)
('10:31:10', -38.0414)
('Episode ', 3403, 'Starts!')
('Collision!', 0)
('10:31:11', -45.816739999999996)
('Episode ', 3404, 'Starts!')
('Collision!', 0)
('10:31:14', -78.84678000000002)
('Episode ', 3405, 'Starts!')
('Collision!', 1)
('10:31:33', -350.26881000000105)
('Episode ', 3406, 'Starts!')
('Collision!', 0)
('10:31:34', -49.308920000000015)
('Episode ', 3407, 'Starts!')
('Collision!', 0)
('10:31:43', -174.85063000000025)
('Episode ', 3408, 'Starts!')
('Collision!', 0)
('10:32:14', -769.392030000004)
(

('Collision!', 0)
('10:42:23', -132.34259000000026)
('Episode ', 3443, 'Starts!')
('Collision!', 0)
('10:42:28', -120.85214999999981)
('Episode ', 3444, 'Starts!')
('Collision!', 0)
('10:42:29', -50.226860000000016)
('Episode ', 3445, 'Starts!')
('Collision!', 0)
('10:42:34', -126.7019899999997)
('Episode ', 3446, 'Starts!')
('Collision!', 0)
('10:43:09', -899.8422300000082)
('Episode ', 3447, 'Starts!')
Goal Reached!
('10:43:32', 9508.255659999999)
('Episode ', 3448, 'Starts!')
('Collision!', 0)
('10:43:32', -37.3528)
('Episode ', 3449, 'Starts!')
('Collision!', 0)
('10:43:36', -83.96168000000003)
('Episode ', 3450, 'Starts!')
('Collision!', 1)
('10:43:47', -238.4341899999994)
('Episode ', 3451, 'Starts!')
('Collision!', 1)
('10:44:00', -253.65124000000102)
('Episode ', 3452, 'Starts!')
('Collision!', 1)
('10:44:26', -568.0575999999979)
('Episode ', 3453, 'Starts!')
('Collision!', 0)
('10:44:27', -43.523189999999985)
('Episode ', 3454, 'Starts!')
('Collision!', 1)
('10:44:44', -396.87

('Collision!', 1)
('10:52:46', -264.20214999999973)
('Episode ', 3489, 'Starts!')
('Collision!', 0)
('10:52:51', -106.30578999999993)
('Episode ', 3490, 'Starts!')
('Collision!', 0)
('10:52:52', -47.949859999999994)
('Episode ', 3491, 'Starts!')
('Collision!', 0)
('10:52:54', -61.23258999999999)
('Episode ', 3492, 'Starts!')
('Collision!', 0)
('10:52:54', -38.6937)
('Episode ', 3493, 'Starts!')
('Collision!', 0)
('10:52:57', -82.56757999999999)
('Episode ', 3494, 'Starts!')
('Collision!', 3)
('10:53:10', -336.1139000000005)
('Episode ', 3495, 'Starts!')
('Collision!', 0)
('10:53:41', -871.9742800000083)
('Episode ', 3496, 'Starts!')
('Collision!', 0)
('10:53:44', -82.22165999999996)
('Episode ', 3497, 'Starts!')
('Collision!', 0)
('10:53:52', -171.51141000000078)
('Episode ', 3498, 'Starts!')
('Collision!', 0)
('10:53:55', -85.31778000000013)
('Episode ', 3499, 'Starts!')
('Collision!', 0)
('10:53:59', -99.00331000000007)
('Episode ', 3500, 'Starts!')
('Collision!', 0)
('10:54:30', -80

('Collision!', 3)
('11:03:58', -483.52783000000056)
('Episode ', 3535, 'Starts!')
('Collision!', 0)
('11:04:23', -576.2574900000008)
('Episode ', 3536, 'Starts!')
('Collision!', 1)
('11:04:51', -770.7578799999956)
('Episode ', 3537, 'Starts!')
('Collision!', 0)
('11:04:54', -76.09962999999993)
('Episode ', 3538, 'Starts!')
('Collision!', 0)
('11:05:01', -156.85412000000036)
('Episode ', 3539, 'Starts!')
('Collision!', 0)
('11:05:06', -100.76344999999995)
('Episode ', 3540, 'Starts!')
('Collision!', 0)
('11:05:07', -58.29695999999999)
('Episode ', 3541, 'Starts!')
('Collision!', 1)
('11:05:25', -381.89063999999985)
('Episode ', 3542, 'Starts!')
('Collision!', 1)
('11:05:39', -256.6895499999995)
('Episode ', 3543, 'Starts!')
('Collision!', 0)
('11:05:45', -130.40638999999965)
('Episode ', 3544, 'Starts!')
('Collision!', 0)
('11:05:47', -55.925560000000004)
('Episode ', 3545, 'Starts!')
('Collision!', 1)
('11:05:52', -94.23952999999999)
('Episode ', 3546, 'Starts!')
('Collision!', 0)
('11

('Collision!', 0)
('11:13:26', -100.78589000000014)
('Episode ', 3581, 'Starts!')
('Collision!', 3)
('11:14:13', -1248.037389999999)
('Episode ', 3582, 'Starts!')
('Collision!', 0)
('11:14:43', -806.1169900000052)
('Episode ', 3583, 'Starts!')
Goal Reached!
('11:15:10', 9262.979509999997)
('Episode ', 3584, 'Starts!')
('Collision!', 2)
('11:16:46', -3159.039360000034)
('Episode ', 3585, 'Starts!')
Goal Reached!
('11:17:31', 8982.868789999991)
('Episode ', 3586, 'Starts!')
('Collision!', 2)
('11:18:58', -3057.5289500000517)
('Episode ', 3587, 'Starts!')
('Collision!', 0)
('11:19:19', -581.5292699999983)
('Episode ', 3588, 'Starts!')
('Collision!', 0)
('11:19:27', -178.95229999999998)
('Episode ', 3589, 'Starts!')
('Collision!', 0)
('11:19:28', -49.723760000000034)
('Episode ', 3590, 'Starts!')
('Collision!', 0)
('11:19:54', -716.5325100000052)
('Episode ', 3591, 'Starts!')
Goal Reached!
('11:20:15', 9483.573060000002)
('Episode ', 3592, 'Starts!')
Goal Reached!
('11:21:48', 6864.2677099